# ⚖️ 감성 분석 모델 비교 실습 (Logistic vs SVM)

In [ ]:

# 필요시 설치
# !pip install pandas scikit-learn


In [ ]:

import pandas as pd

df = pd.read_csv("36000_reviews_label.csv")
df = df[['sentence', 'label']].dropna()
df = df[df['label'].isin(['긍정', '부정', '중립'])]


In [3]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['sentence'], df['label'], test_size=0.2, random_state=42
)


In [4]:

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2), min_df=5)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [5]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

logi = LogisticRegression(max_iter=1000)
logi.fit(X_train_vec, y_train)
logi_pred = logi.predict(X_test_vec)

print("📌 Logistic Regression 결과:")
print(classification_report(y_test, logi_pred, digits=3))


📌 Logistic Regression 결과:
              precision    recall  f1-score   support

          긍정      0.921     0.975     0.947      5604
          모호      0.624     0.532     0.574      1107
          부정      0.775     0.579     0.663       649

    accuracy                          0.873      7360
   macro avg      0.773     0.695     0.728      7360
weighted avg      0.864     0.873     0.866      7360



In [6]:

from sklearn.svm import LinearSVC

svm = LinearSVC()
svm.fit(X_train_vec, y_train)
svm_pred = svm.predict(X_test_vec)

print("📌 SVM 결과:")
print(classification_report(y_test, svm_pred, digits=3))


📌 SVM 결과:
              precision    recall  f1-score   support

          긍정      0.933     0.964     0.948      5604
          모호      0.619     0.537     0.575      1107
          부정      0.710     0.670     0.689       649

    accuracy                          0.874      7360
   macro avg      0.754     0.724     0.738      7360
weighted avg      0.866     0.874     0.869      7360

